In [11]:
import pandas as pd

## Muestra del catalogo

In [12]:
df = pd.read_excel('../catalogo/catalogoCompletoINEGI.xlsx')
df.tail()

,Variables,Claves,FechaActualizacion,Nivel1
96158,Sector externo > Balanza comercial de mercancí...,133938,Fecha de última actualización: 2023-12-14,Sector externo
96159,Sector externo > Balanza comercial de mercancí...,133939,Fecha de última actualización: 2023-12-14,Sector externo
96160,Sector externo > Balanza comercial de mercancí...,133940,Fecha de última actualización: 2023-12-14,Sector externo
96161,Sector externo > Balanza comercial de mercancí...,133941,Fecha de última actualización: 2023-12-14,Sector externo
96162,Minería > Volumen de producción minera por pri...,161983,Fecha de última actualización: 2023-12-15,Minería


In [13]:
sample5 = df.sample(5, random_state=42)
sample5

,Variables,Claves,FechaActualizacion,Nivel1
62633,Manufacturas > Encuesta anual de la industria ...,692895,Fecha de última actualización: 2022-12-06,Manufacturas
30650,Manufacturas > Encuesta mensual de la industri...,648292,Fecha de última actualización: 2023-05-12,Manufacturas
71412,Manufacturas > Encuesta mensual de la industri...,703200,Fecha de última actualización: 2023-12-14,Manufacturas
64117,Manufacturas > Encuesta anual de la industria ...,694379,Fecha de última actualización: 2022-12-06,Manufacturas
22176,Manufacturas > Encuesta mensual de la industri...,639372,Fecha de última actualización: 2023-05-12,Manufacturas


In [15]:
sample5.to_excel("../catalogo/muestra5.xlsx")

In [22]:
sample5[["Variables"]].to_excel("../catalogo/muestra5-rutas.xlsx", index=False)

In [24]:
sample5[["Claves"]].to_excel("../catalogo/muestra5-claves.xlsx", index=False)

## Construir los catalogos por rutas y claves

In [ ]:
import os
import sys
import pandas as pd 
from INEGIpy import Indicadores

catalogo_path: str = "catalogo/catalogoCompletoINEGI.xlsx"
catalogo: pd.DataFrame = pd.read_excel(catalogo_path)

def construir_catalogo(formato: str):
  if formato.strip() == "Rutas":
    columna_formato = "Variables"
  elif formato.strip() == "Claves":
    columna_formato = "Claves"
  return catalogo[["Variables", "Claves"]].set_index(columna_formato).squeeze()

## Ejemplo de uso

In [40]:
def obtener_serie(ruta_archivo: str, formato:str, token:str = "f6a7b69c-5c48-bf0c-b191-5ca98c6a6cc0"):
  # Tomamos siempre la primera columna
  variables_usuario: pd.Series = pd.read_excel(ruta_archivo).iloc[:,0]
  catalogo_se: pd.Series = construir_catalogo(formato)
  
  # Se instancia la interfaz que se counicara con INEGI
  inegi = Indicadores(token)  

  if formato == "Rutas":
    #Para cada variable tendremos que sacar su clave y nombre de la variable
    claves_variables =  variables_usuario.apply(lambda x: catalogo_se[x])
    nombres_variables = variables_usuario.apply(lambda x: x.split(">")[-1])

    # Se obtiene la serie a partir de la API
    variables_df = inegi.obtener_df(indicadores=claves_variables, nombres=nombres_variables)
  
  elif formato == "Claves":
    claves_variables =  variables_usuario
    nombres_variables = variables_usuario.apply(lambda x: catalogo_se[x].split(">")[-1])
    variables_df = inegi.obtener_df(indicadores=claves_variables, nombres=nombres_variables)
  return variables_df

In [19]:
rutas_examples = pd.read_excel("catalogo/muestra5-rutas.xlsx")
rutas_examples

,Variables
0,Manufacturas > Encuesta anual de la industria ...
1,Manufacturas > Encuesta mensual de la industri...
2,Manufacturas > Encuesta mensual de la industri...
3,Manufacturas > Encuesta anual de la industria ...
4,Manufacturas > Encuesta mensual de la industri...


In [42]:
obtener_serie('catalogo/muestra5-rutas.xlsx', "Rutas")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/INEGIpy/_indicadores.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.set_index(pd.to_datetime(df.fechas),inplace=True, drop=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/INEGIpy/_indicadores.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.set_index(pd.to_datetime(df.fechas),inplace=True, drop=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/INEGIpy/_indicadores.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify

Variables,311211 Beneficio del arroz (Miles de pesos),Valor total de la clase (Miles de pesos corrientes),Total 3121 Industria de las bebidas (Número de personas),"316999 Fabricación de otros productos de cuero, piel y materiales sucedáneos (Miles de pesos)",332320 Fabricación de productos de herrería (Miles de pesos corrientes)
fechas,,,,,
2013-01-01,76.0,54599.0,NaN,1196.0,231972.0
2013-02-01,NaN,40839.0,NaN,NaN,245660.0
2013-03-01,NaN,45826.0,NaN,NaN,240156.0
2013-04-01,NaN,57390.0,NaN,NaN,236798.0
2013-05-01,NaN,59118.0,NaN,NaN,255008.0
...,...,...,...,...,...
2023-06-01,NaN,NaN,151922.0,NaN,NaN
2023-07-01,NaN,NaN,153384.0,NaN,NaN
2023-08-01,NaN,NaN,154333.0,NaN,NaN


In [27]:
claves_examples = pd.read_excel("catalogo/muestra5-claves.xlsx")
claves_examples

,Claves
0,692895
1,648292
2,703200
3,694379
4,639372


In [43]:
df = obtener_serie('catalogo/muestra5-claves.xlsx', "Claves")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/INEGIpy/_indicadores.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.set_index(pd.to_datetime(df.fechas),inplace=True, drop=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/INEGIpy/_indicadores.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.set_index(pd.to_datetime(df.fechas),inplace=True, drop=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/INEGIpy/_indicadores.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify

In [54]:
df

Claves,311211 Beneficio del arroz (Miles de pesos),Valor total de la clase (Miles de pesos corrientes),Total 3121 Industria de las bebidas (Número de personas),"316999 Fabricación de otros productos de cuero, piel y materiales sucedáneos (Miles de pesos)",332320 Fabricación de productos de herrería (Miles de pesos corrientes)
fechas,,,,,
2013-01-01,76.0,54599.0,NaN,1196.0,231972.0
2013-02-01,NaN,40839.0,NaN,NaN,245660.0
2013-03-01,NaN,45826.0,NaN,NaN,240156.0
2013-04-01,NaN,57390.0,NaN,NaN,236798.0
2013-05-01,NaN,59118.0,NaN,NaN,255008.0
...,...,...,...,...,...
2023-06-01,NaN,NaN,151922.0,NaN,NaN
2023-07-01,NaN,NaN,153384.0,NaN,NaN
2023-08-01,NaN,NaN,154333.0,NaN,NaN


In [56]:
df.set_index(pd.to_datetime(df.index.get_level_values("fechas")),inplace=True, drop=True)

In [57]:
df

Claves,311211 Beneficio del arroz (Miles de pesos),Valor total de la clase (Miles de pesos corrientes),Total 3121 Industria de las bebidas (Número de personas),"316999 Fabricación de otros productos de cuero, piel y materiales sucedáneos (Miles de pesos)",332320 Fabricación de productos de herrería (Miles de pesos corrientes)
fechas,,,,,
2013-01-01,76.0,54599.0,NaN,1196.0,231972.0
2013-02-01,NaN,40839.0,NaN,NaN,245660.0
2013-03-01,NaN,45826.0,NaN,NaN,240156.0
2013-04-01,NaN,57390.0,NaN,NaN,236798.0
2013-05-01,NaN,59118.0,NaN,NaN,255008.0
...,...,...,...,...,...
2023-06-01,NaN,NaN,151922.0,NaN,NaN
2023-07-01,NaN,NaN,153384.0,NaN,NaN
2023-08-01,NaN,NaN,154333.0,NaN,NaN


In [55]:
df.index.get_level_values("fechas")

DatetimeIndex(['2013-01-01', '2013-02-01', '2013-03-01', '2013-04-01',
               '2013-05-01', '2013-06-01', '2013-07-01', '2013-08-01',
               '2013-09-01', '2013-10-01',
               ...
               '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01',
               '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01',
               '2023-09-01', '2023-10-01'],
              dtype='datetime64[ns]', name='fechas', length=130, freq='MS')